In [1]:
import numpy as np

Batch = 12
N = 197
d = 64

S = np.random.random(size=(Batch, N, N))
V = np.random.random(size=(Batch, N, d))

def increasing_softmax_then_matmul(S, V):
  acc = np.zeros(shape=(Batch, N, d))
  pre_max = -10000
  pre_sum = 0
  for i  in range(N):
    s_i = S[:,i:i+1]
    # import pdb;pdb.set_trace()
    cur_max = max(pre_max, np.max(s_i))
    pre_sum *= np.sum(np.exp(pre_max - cur_max))
    cur_sum = pre_sum + np.sum(np.exp(s_i - cur_max))
    score = np.exp(s_i - cur_max) / cur_sum
    scale = pre_sum / cur_sum
    acc *= scale
    acc += (score.transpose(0,2,1) @ V[:,i:i+1,:])
    pre_max = cur_max
    pre_sum = cur_sum
  return acc
    
def softmax_then_matmul(S, V):
  row_max = np.amax(S, -1)[:,:,None]
  row_sum = np.sum(np.exp(S - row_max), -1)[:,:,None]
  import pdb;pdb.set_trace()
  softmax_s = np.exp(S-row_max) / row_sum
  return np.matmul(softmax_s, V)
  
expected = softmax_then_matmul(S, V)
actual = increasing_softmax_then_matmul(S, V)
np.allclose(expected, actual)

> /tmp/ipykernel_1708143/3549042524.py(32)softmax_then_matmul()
     30   row_sum = np.sum(np.exp(S - row_max), -1)[:,:,None]
     31   import pdb;pdb.set_trace()
---> 32   softmax_s = np.exp(S-row_max) / row_sum
     33   return np.matmul(softmax_s, V)
     34 



ipdb>  q


In [3]:
import torch
import torch.nn.functional as F

x = torch.randn((12,197,197))

In [2]:
%time softmax_then_matmul(S, V)

CPU times: user 70 µs, sys: 120 µs, total: 190 µs
Wall time: 150 µs


array([[0.69450367, 0.64144421]])

In [3]:
%time increasing_softmax_then_matmul(S, V)

CPU times: user 86 µs, sys: 146 µs, total: 232 µs
Wall time: 241 µs


array([[0.69450367, 0.64144421]])